<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image_predictions.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image_predictions.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

----

# Model Diagnostics - Custom Metrics Demo

* Measuring model quality is critical to efficiently building models. It is important that the metrics used to measure model quality closely align with the business objectives for the model. Otherwise, slight changes in model quality, as they related to these core objectives, are lost to noise. Custom metrics enables users to measure model quality in terms of their exact business goals. By incorporating custom metrics into workflows, users can:
    * Iterate faster
    * Measure and report on model quality
    * Understand marginal value of additional labels and modeling efforts


## Environment setup

In [ ]:
!pip install -q 'labelbox[data]'

In [ ]:
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, MALPredictionImport, LabelImport
from labelbox.data.serialization import NDJsonConverter
from labelbox.schema.media_type import MediaType
from labelbox.schema.quality_mode import QualityMode
from labelbox.data.metrics.group import get_label_pairs
from labelbox.data.metrics import feature_miou_metric, feature_confusion_matrix_metric
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer, RelationshipAnnotation, Relationship
)
import uuid
import numpy as np
from labelbox.schema.queue_mode import QueueMode

## Replace with your API Key
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
API_KEY = ""

client = Client(API_KEY)

## Supported Predictions

In [ ]:
########### Radio Classification ###########

# Python annotation
radio_prediction = ClassificationAnnotation(
    name="radio_question",
    value=Radio(answer = ClassificationAnswer(name = "first_radio_answer", confidence=0.5))
)

# NDJSON
radio_prediction_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'first_radio_answer', 'confidence': 0.5}
}

########## Nested Classifications are only supported with NDJSON tools ##########

nested_radio_prediction_ndjson = {
  "name": "nested_radio_question",
  "confidence": 0.5 ,
  "answer": { "name": "first_radio_answer", "confidence": 0.5 },
      "classifications" : [
      {
        "name": "sub_radio_question",
        "answer": {"name": "first_sub_radio_answer", "confidence": 0.5 }
      }
    ]
}


nested_checklist_prediction_ndjson = {
  "name": "nested_checklist_question",
  "confidence": 0.5 ,
  "answer": [{
      "name": "first_checklist_answer",
      "confidence": 0.5,
      "classifications" : [
        {
          "name": "sub_checklist_question",
          "answer": {"name": "first_sub_checklist_answer", "confidence": 0.5 }
        }
      ]
  }]
}

############ Checklist ############

# Python Annotations
checklist_prediction = ClassificationAnnotation(
  name="checklist_question", # must match your ontology feature's name
  value=Checklist(
      answer = [
        ClassificationAnswer(
            name = "first_checklist_answer",
            confidence=0.5
        ),
        ClassificationAnswer(
            name = "second_checklist_answer",
            confidence=0.5
        )
      ]
    )
 )

# NDJSON
checklist_prediction_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer' , 'confidence': 0.5},
    {'name': 'second_checklist_answer', 'confidence': 0.5}
  ]
}

####### Bounding box #######


# Python Annotation
bbox_prediction = ObjectAnnotation(
  name = "bounding_box",  # must match your ontology feature's name
  confidence=0.5,
  value=Rectangle(
        start=Point(x=977, y=1690), # Top left
        end=Point(x=330, y=225), # Bottom right
    ),

)

#NDJSON
bbox_prediction_ndjson = {
  'name': 'bounding_box',
  'confidence': 0.5,
  'bbox': {
          "top": 500,
          "left": 1190,
          "height": 230,
          "width": 225
      }
}

####### Bounding box with nested classification #######
bbox_with_radio_subclass_prediction = ObjectAnnotation(
    name="bbox_with_radio_subclass",
    confidence=0.5, # must match your ontology feature's name
    value=Rectangle(
        start=Point(x=433, y=341), # Top left
        end=Point(x=291, y=230), # Bottom right
    ),
    classifications=[
    	ClassificationAnnotation(
        	name="sub_radio_question",
      		value=Radio(answer=ClassificationAnswer(name="first_sub_radio_answer", confidence=0.5))
    )
  ]
)


## NDJSON
bbox_with_radio_subclass_prediction_ndjson = {
    "name": "bbox_with_radio_subclass",
    "confidence": 0.5,
    "classifications": [{
        "name": "sub_radio_question",
        "confidence": 0.5,
        "answer":
            { "name":"first_sub_radio_answer", "confidence": 0.5}

    }],
    "bbox": {
          "top": 533,
          "left": 641,
          "height": 291,
          "width": 330
        }
}


####### Bounding box with nested free text #######

## NDJSON
bbox_with_free_text_subclass_prediction_ndjson = {
    "name": "bbox_with_free_text_subclass",
    "confidence": 0.5,
    "classifications": [{
        "name": "text",
        "confidence": 0.5,
        'answer': 'sample text',

    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}

########## Polygon ##########
# Python Anotation
polygon_prediction = ObjectAnnotation(
  name = "polygon",  # must match your ontology feature's name
  confidence = 0.5,
  value=Polygon( # Coordinates for the verticies of your polygon
        points=[Point(x=1489.581,y=183.934),Point(x=2278.306,y=256.885),Point(x=2428.197,y=200.437),Point(x=2560.0,y=335.419),
                Point(x=2557.386,y=503.165),Point(x=2320.596,y=503.103),Point(x=2156.083, y=628.943),Point(x=2161.111,y=785.519),
                Point(x=2002.115, y=894.647),Point(x=1838.456,y=877.874),Point(x=1436.53,y=874.636),Point(x=1411.403,y=758.579),
                Point(x=1353.853,y=751.74),Point(x=1345.264, y=453.461),Point(x=1426.011,y=421.129)]
    ),
)

polygon_prediction_target = ObjectAnnotation(
  name = "polygon",  # must match your ontology feature's name
  confidence = 0.5,
  value=Polygon( # Coordinates for the verticies of your polygon
        points=[Point(x=1089.581,y=203.934),Point(x=1878.306,y=276.885),Point(x=2028.197,y=220.437),Point(x=2160.0,y=355.419),
                Point(x=2157.386,y=523.165),Point(x=2020.596,y=523.103),Point(x=1756.083, y=648.943),Point(x=1761.111,y=805.519),
                Point(x=1602.115, y=914.647),Point(x=1438.456,y=897.874),Point(x=1036.53,y=894.636),Point(x=1011.403,y=778.579),
                Point(x=953.853,y=771.74),Point(x=945.264, y=473.461),Point(x=926.011,y=441.129)]
    ),
)

polygon_relationship = RelationshipAnnotation(
    name="relationship",
    value=Relationship(
        source=polygon_prediction,
        target=polygon_prediction_target,
        type=Relationship.Type.UNIDIRECTIONAL,
    ))


# NDJSON

polygon_prediction_ndjson = {
  'name': 'polygon',
  'confidence': 0.5,
  'polygon': [
    {'x': 1489.581, 'y': 183.934},
    {'x': 2278.306, 'y': 256.885},
    {'x': 2428.197, 'y': 200.437},
    {'x': 2560.0, 'y': 335.419},
    {'x': 2557.386, 'y': 503.165},
    {'x': 2320.596, 'y': 503.103},
    {'x': 2156.083, 'y': 628.943},
    {'x': 2161.111, 'y': 785.519},
    {'x': 2002.115, 'y': 894.647},
    {'x': 1838.456, 'y': 877.874},
    {'x': 1436.53, 'y': 874.636},
    {'x': 1411.403, 'y': 758.579},
    {'x': 1353.853, 'y': 751.74},
    {'x': 1345.264, 'y': 453.461},
    {'x': 1426.011, 'y': 421.129},
    {'x': 1489.581, 'y': 183.934}
  ]
}

####### Free text #######
# Confidence is not supported for text prediction
# Python annotation
text_annotation = ClassificationAnnotation(
  name="free_text",  # must match your ontology feature's name
  value=Text(answer="sample text")
)

# NDJSON
text_annotation_ndjson = {
  'name': 'free_text',
  'answer': 'sample text',
}

######### Segmentation mask #########

# Python
# Identifying what values in the numpy array correspond to the mask annotation
color = (0, 0, 0)

# convert a polygon to mask
im_height, im_width = 100,100 #need to provide the height and width of image.
mask_data = MaskData(arr=
                     polygon_prediction.value.draw(height=im_height,width=im_width,color=color))

# convert a 2D array to 3D array
arr_2d = np.zeros((100,100), dtype='uint8')
mask_data = MaskData.from_2D_arr(arr_2d)

# a 3D array where 3rd axis is RGB values.
mask_data = MaskData(arr= np.zeros([400,450,3],dtype='uint8'))

mask_prediction = ObjectAnnotation(
  name = "mask", # must match your ontology feature's name
  confidence=0.5,
  value=Mask(mask=mask_data, color=color),
)

mask_prediction_ndjson = {
  "name": "mask",
  "confidence": 0.5,
  "classifications": [],
  "mask": {"instanceURI": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/raster_seg.png",
  "colorRGB": (255, 255, 255)}
}

######## Point ########

# Python Annotation
point_prediction = ObjectAnnotation(
  name = "point",  # must match your ontology feature's name
  confidence=0.5,
  value = Point(x=1166.606, y=1441.768),
)


# NDJSON
point_prediction_ndjson = {
  'name': 'point',
  'confidence': 0.5,
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

###### Polyline ######


# Python Annotation

polyline_prediction = ObjectAnnotation(
  name = "polyline", # must match your ontology feature's name
  confidence=0.5, ## Not supported for python annotation tools
  value=Line( # Coordinates for the keypoints in your polyline
        points=[Point(x=2534.353, y=249.471),Point(x=2429.492, y=182.092),Point(x=2294.322, y=221.962),Point(x=2224.491, y=180.463),Point(x=2136.123, y=204.716),
                Point(x=1712.247, y=173.949),Point(x=1703.838, y=84.438),Point(x=1579.772, y=82.61),Point(x=1583.442, y=167.552),
                Point(x=1478.869, y=164.903),Point(x=1418.941, y=318.149),Point(x=1243.128, y=400.815),Point(x=1022.067, y=319.007),
                Point(x=892.367, y=379.216),Point(x=670.273, y=364.408),Point(x=613.114, y=288.16),Point(x=377.559, y=238.251),
                Point(x=368.087, y=185.064),Point(x=246.557, y=167.286),Point(x=236.648, y=285.61),Point(x=90.929, y=326.412)]
    ),
)

# NDJSON
polyline_prediction_ndjson = {
  'name': 'polyline',
  'confidence':0.5,
  'classifications': [],
  'line': [
    {'x': 2534.353, 'y': 249.471},
    {'x': 2429.492, 'y': 182.092},
    {'x': 2294.322, 'y': 221.962},
    {'x': 2224.491, 'y': 180.463},
    {'x': 2136.123, 'y': 204.716},
    {'x': 1712.247, 'y': 173.949},
    {'x': 1703.838, 'y': 84.438},
    {'x': 1579.772, 'y': 82.61},
    {'x': 1583.442, 'y': 167.552},
    {'x': 1478.869, 'y': 164.903},
    {'x': 1418.941, 'y': 318.149},
    {'x': 1243.128, 'y': 400.815},
    {'x': 1022.067, 'y': 319.007},
    {'x': 892.367, 'y': 379.216},
    {'x': 670.273, 'y': 364.408},
    {'x': 613.114, 'y': 288.16},
    {'x': 377.559, 'y': 238.251},
    {'x': 368.087, 'y': 185.064},
    {'x': 246.557, 'y': 167.286},
    {'x': 236.648, 'y': 285.61},
    {'x': 90.929, 'y': 326.412}
  ]
}



## Step 1: Import data rows into Catalog

In [ ]:
NUM_DATA_ROWS = 1

# send a sample image as batch to the project
test_img_urls = [{
    "row_data": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key": str(uuid.uuid4())
} for i in range(0, NUM_DATA_ROWS)]
dataset = client.create_dataset(name="image_prediction_demo", iam_integration=None)
data_rows = dataset.create_data_rows(test_img_urls)
print(data_rows)

## Step 2: Create/select an Ontology for your model predictions
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the name/instructions fields in your annotations to ensure the correct feature schemas are matched.


In [ ]:
ontology_builder = OntologyBuilder(
  classifications=[ # List of Classification objects
    Classification( # Radio classification given the name "text" with two options: "first_radio_answer" and "second_radio_answer"
      class_type=Classification.Type.RADIO,
      name="radio_question",
      options=[
        Option(value="first_radio_answer"),
        Option(value="second_radio_answer")
      ]
    ),
    Classification( # Checklist classification given the name "text" with two options: "first_checklist_answer" and "second_checklist_answer"
      class_type=Classification.Type.CHECKLIST,
      name="checklist_question",
      options=[
        Option(value="first_checklist_answer"),
        Option(value="second_checklist_answer")
      ]
    ),
    Classification( # Text classification given the name "text"
      class_type=Classification.Type.TEXT,
      name="free_text"
    ),
    Classification(
        class_type=Classification.Type.RADIO,
        name="nested_radio_question",
        options=[
            Option("first_radio_answer",
                options=[
                    Classification(
                        class_type=Classification.Type.RADIO,
                        name="sub_radio_question",
                        options=[Option("first_sub_radio_answer")]
                    )
                ]
            )
          ]
        ),
    Classification(
      class_type=Classification.Type.CHECKLIST,
      name="nested_checklist_question",
      options=[
          Option("first_checklist_answer",
            options=[
              Classification(
                  class_type=Classification.Type.CHECKLIST,
                  name="sub_checklist_question",
                  options=[Option("first_sub_checklist_answer")]
              )
          ]
        )
      ]
    ),
  ],
  tools=[ # List of Tool objects
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX,
      name="bounding_box"),
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX,
      name="bbox_with_radio_subclass",
      classifications=[
            Classification(
                class_type=Classification.Type.RADIO,
                name="sub_radio_question",
                options=[
                  Option(value="first_sub_radio_answer")
                ]
              ),
        ]
      ),
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX,
      name="bbox_with_free_text_subclassification",
      classifications=[
            Classification(
                class_type=Classification.Type.TEXT,
                name="sub_text_classification",
              ),
        ]
      ),
    Tool( # Polygon tool given the name "polygon"
      tool=Tool.Type.POLYGON,
      name="polygon"),
    Tool( # Segmentation mask tool given the name "mask"
      tool=Tool.Type.RASTER_SEGMENTATION,
      name="mask"),
 	  Tool( # Point tool given the name "point"
      tool=Tool.Type.POINT,
      name="point"),
    Tool( # Polyline tool given the name "line"
      tool=Tool.Type.LINE,
      name="polyline")]
)

ontology = client.create_ontology("Image Prediction Import Demo", ontology_builder.asdict(), media_type=MediaType.Image)

## Step 3: Create a Model and Model Run

In [ ]:
# create Model
model = client.create_model(name="with_aggregated_custom_metrics" + str(uuid.uuid4()),
                            ontology_id=ontology.uid)
# create Model Run
model_run = model.create_model_run("iteration 1")

## Step 4: Send data rows to the Model Run

In [ ]:
model_run.upsert_data_rows(global_keys=[data_row.global_key for data_row in dataset.export_data_rows()])

## Step 5. Create the predictions payload

Create the prediction payload using the snippets of code in ***Supported Predictions*** section.

The resulting label_ndjson should have exactly the same content for predictions that are supported by both (with exception of the uuid strings that are generated)

In [ ]:
# Create a Label for predictions
label_predictions = [Label(
    data=ImageData(uid=data_row.uid),
    annotations = [
      radio_prediction,
      checklist_prediction,
      bbox_prediction,
      bbox_with_radio_subclass_prediction,
      polygon_prediction,
      # mask_prediction,
      point_prediction,
      text_annotation,
    ]
) for data_row in dataset.export_data_rows()]

# Convert the prediction label from a Labelbox class object to the underlying NDJSON format required for upload - uploads can be directly built in this syntax as well
ndjson_prediction = list(NDJsonConverter.serialize(label_predictions))

If using NDJSON

## Custom metrics

In [ ]:
# Special case for custom feature metrics

# NDJSON
radio_prediction_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'first_radio_answer', 'confidence': 0.5,         'customMetrics': [
            { 'name': 'iou', 'value': 0.1 },
            { 'name': 'f1', 'value': 0.33 },
            { 'name': 'precision', 'value': 0.55 },
            { 'name': 'recall', 'value': 0.33 },
            { 'name': 'tagsCount', 'value': 43 },
            { 'name': 'metric_with_a_very_long_name', 'value': 0.334332 }
          ]}
}

########## Nested Classifications are only supported with NDJSON tools ##########

nested_radio_prediction_ndjson = {
  "name": "nested_radio_question",
  "confidence": 0.5 ,
  "answer": { "name": "first_radio_answer", "confidence": 0.5,         'customMetrics': [
            { 'name': 'iou', 'value': 0.5 },
            { 'name': 'f1', 'value': 0.33 },
            { 'name': 'precision', 'value': 0.55 },
            { 'name': 'recall', 'value': 0.33 },
            { 'name': 'tagsCount', 'value': 43 },
            { 'name': 'metric_with_a_very_long_name', 'value': 0.334332 }
          ] },
      "classifications" : [
      {
        "name": "sub_radio_question",
        "answer": {"name": "first_sub_radio_answer", "confidence": 0.5,         'customMetrics': [
            { 'name': 'iou', 'value': 0.5 },
            { 'name': 'f1', 'value': 0.33 },
            { 'name': 'precision', 'value': 0.55 },
            { 'name': 'recall', 'value': 0.33 },
            { 'name': 'tagsCount', 'value': 43 },
            { 'name': 'metric_with_a_very_long_name', 'value': 0.334332 }
          ] }
      }
    ]
}


nested_checklist_prediction_ndjson = {
  "name": "nested_checklist_question",
  "confidence": 0.5 ,
  "answer": [{
      "name": "first_checklist_answer",
      "confidence": 0.5,
              'customMetrics': [
            { 'name': 'iou', 'value': 0.5 },
            { 'name': 'f1', 'value': 0.33 },
            { 'name': 'precision', 'value': 0.55 },
            { 'name': 'recall', 'value': 0.33 },
            { 'name': 'tagsCount', 'value': 43 },
            { 'name': 'metric_with_a_very_long_name', 'value': 0.334332 }
          ],
      "classifications" : [
        {
          "name": "sub_checklist_question",
          "answer": {"name": "first_sub_checklist_answer", "confidence": 0.5,         'customMetrics': [
            { 'name': 'iou', 'value': 0.5 },
            { 'name': 'f1', 'value': 0.33 },
            { 'name': 'precision', 'value': 0.55 },
            { 'name': 'recall', 'value': 0.33 },
            { 'name': 'tagsCount', 'value': 43 },
            { 'name': 'metric_with_a_very_long_name', 'value': 0.334332 }
          ] }
        }
      ]
  }]
}

############ Checklist ############
checklist_prediction_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer' , 'confidence': 0.5,         'customMetrics': [
            { 'name': 'iou', 'value': 0.5 },
            { 'name': 'f1', 'value': 0.33 },
            { 'name': 'precision', 'value': 0.55 },
            { 'name': 'recall', 'value': 0.33 },
            { 'name': 'tagsCount', 'value': 43 },
            { 'name': 'metric_with_a_very_long_name', 'value': 0.334332 }
          ]},
    {'name': 'second_checklist_answer', 'confidence': 0.5,         'customMetrics': [
            { 'name': 'iou', 'value': 0.5 },
            { 'name': 'f1', 'value': 0.33 },
            { 'name': 'precision', 'value': 0.55 },
            { 'name': 'recall', 'value': 0.33 },
            { 'name': 'tagsCount', 'value': 43 },
            { 'name': 'metric_with_a_very_long_name', 'value': 0.334332 }
          ]}
  ]
}
all_annotations = [
      radio_prediction_ndjson,
      nested_radio_prediction_ndjson,
      checklist_prediction_ndjson,
      nested_checklist_prediction_ndjson,
      ]

three_k_annotations = all_annotations

ndjson_prediction_method2 = []
dr_ids = [data_row.uid for data_row in dataset.export_data_rows()]
for data_row_id in dr_ids:
  for annot in three_k_annotations:
    ndjson_prediction_method2.append({
        **annot,
        'uuid': str(uuid.uuid4()),
        'dataRow': {'id': data_row_id },
    })

## OCR special case

In [ ]:
# Special case for OCR

bbox_with_free_text_subclass_prediction_ndjson1 = {
    "name": "bbox_with_free_text_subclassification",
    "confidence": 0.5,
    "classifications": [{
        "name": "sub_text_classification",
        "confidence": 0.5,
        'answer': 'aaaaaaaa - matching nicely',

    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}
bbox_with_free_text_subclass_prediction_ndjson2 = {
    "name": "bbox_with_free_text_subclassification",
    "confidence": 0.5,
    "classifications": [{
        "name": "sub_text_classification",
        "confidence": 0.5,
        'answer': 'zzzzzzz',

    }],
    "bbox": {
          "top": 913,
          "left": 521,
          "height": 50,
          "width": 50
        }
}
all_annotations = [
      bbox_with_free_text_subclass_prediction_ndjson1,
      bbox_with_free_text_subclass_prediction_ndjson2,
      ]

three_k_annotations = []
three_k_annotations.extend(all_annotations)

ndjson_prediction_method2 = []
dr_ids = [data_row.uid for data_row in dataset.export_data_rows()]
for data_row_id in dr_ids:
  for annot in three_k_annotations:
    ndjson_prediction_method2.append({
        **annot,
        'uuid': str(uuid.uuid4()),
        'dataRow': {'id': data_row_id },
    })

In [ ]:

all_annotations = [
      radio_prediction_ndjson,
      checklist_prediction_ndjson,
      bbox_prediction_ndjson,
      bbox_with_radio_subclass_prediction_ndjson,
      polygon_prediction_ndjson,
      # mask_prediction_ndjson,
      point_prediction_ndjson,
      polyline_prediction_ndjson,
      text_annotation_ndjson,
      nested_radio_prediction_ndjson,
      nested_checklist_prediction_ndjson
      ]

# three_k_annotations = bbox_predictions
# three_k_annotations.extend(all_annotations)
three_k_annotations = all_annotations

ndjson_prediction_method2 = []
dr_ids = [data_row.uid for data_row in dataset.export_data_rows()]
for data_row_id in dr_ids:
  for annot in three_k_annotations:
    ndjson_prediction_method2.append({
        **annot,
        'uuid': str(uuid.uuid4()),
        'dataRow': {'id': data_row_id }
    })

## Step 6. Upload the predictions payload to the Model Run

In [ ]:
# Upload the prediction label to the Model Run
upload_job_prediction = model_run.add_predictions(
    name="prediction_upload_job"+str(uuid.uuid4()),
    predictions=ndjson_prediction_method2)

# Errors will appear for prediction uploads that failed.
print("Errors:",  upload_job_prediction.errors)

## Step 7: Send annotations to a model run
To visualize both annotations and predictions in the model run we will create a project with ground truth annotations.
To send annotations to a Model Run, we must first import them into a project, create a label payload and then send them to the Model Run.

##### 7.1. Create a labelbox project

In [ ]:
# Create a Labelbox project
project = client.create_project(name="image_prediction_many_kinds",
                                    quality_mode=QualityMode.Benchmark,
                                    media_type=MediaType.Image)
project.setup_editor(ontology)

##### 7.2. Create a batch to send to the project

In [ ]:
project.create_batch(
  "batch_predictions_demo1", # Each batch in a project must have a unique name
  [data_row.uid for data_row in dataset.export_data_rows()], # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

##### 7.3 Create the annotations payload

In [ ]:
########### Annotations ###########
radio_annotation_ndjson = {
  "name": "radio_question",
  "answer": {"name": "first_radio_answer"}
}

nested_radio_annotation_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer"},
  "classifications" : [
   {"name": "sub_radio_question", "answer": {"name": "first_sub_radio_answer"}}
   ]
}

checklist_annotation_ndjson = {
  "name": "checklist_question",
  "answer": [
    {"name": "first_checklist_answer"},
    {"name": "second_checklist_answer"}
  ]
}

bbox_annotation_ndjson = {
  "name": "bounding_box",
  "bbox": {
          "top": 877,
          "left": 1490,
          "height": 130,
          "width": 125
      }
}

bbox_with_radio_subclass_ndjson = {
    "name": "bbox_with_radio_subclass",
    "classifications": [{
        "name": "sub_radio_question",
        "answer":
            { "name":"first_sub_radio_answer" }

    }],
    "bbox": {
          "top": 533,
          "left": 841,
          "height": 191,
          "width": 230
        }
}

polygon_annotation_ndjson = {
  "name": "polygon",
  "polygon": [
    {"x": 1489.581, "y": 183.934},
    {"x": 2278.306, "y": 256.885},
    {"x": 2428.197, "y": 200.437},
    {"x": 2560.0, "y": 335.419},
    {"x": 2557.386, "y": 503.165},
    {"x": 2320.596, "y": 503.103},
    {"x": 2156.083, "y": 628.943},
    {"x": 2161.111, "y": 785.519},
    {"x": 2002.115, "y": 894.647},
    {"x": 1838.456, "y": 877.874},
    {"x": 1436.53, "y": 874.636},
    {"x": 1411.403, "y": 758.579},
    {"x": 1353.853, "y": 751.74},
    {"x": 1345.264, "y": 453.461},
    {"x": 1426.011, "y": 421.129},
    {"x": 1489.581, "y": 183.934}
  ]
}

mask_annotation_ndjson = {
  "name": "mask",
  "classifications": [],
  "mask": {"instanceURI": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/raster_seg.png",
  "colorRGB": (0, 0, 0)}
}


point_annotation_ndjson = {
  "name": "point",
  "classifications": [],
  "point": {"x": 1166.606, "y": 1441.768}
}

point_annotation_ndjson = {
  "name": "point",
  "classifications": [],
  "point": {"x": 1166.606, "y": 1441.768}
}

polyline_annotation_ndjson = {
  "name": "polyline",
  "classifications": [],
  "line": [
    {"x": 2534.353, "y": 249.471},
    {"x": 2429.492, "y": 182.092},
    {"x": 2294.322, "y": 221.962},
    {"x": 2224.491, "y": 180.463},
    {"x": 2136.123, "y": 204.716},
    {"x": 1712.247, "y": 173.949},
    {"x": 1703.838, "y": 84.438},
    {"x": 1579.772, "y": 82.61},
    {"x": 1583.442, "y": 167.552},
    {"x": 1478.869, "y": 164.903},
    {"x": 1418.941, "y": 318.149},
    {"x": 1243.128, "y": 400.815},
    {"x": 1022.067, "y": 319.007},
    {"x": 892.367, "y": 379.216},
    {"x": 670.273, "y": 364.408},
    {"x": 613.114, "y": 288.16},
    {"x": 377.559, "y": 238.251},
    {"x": 368.087, "y": 185.064},
    {"x": 246.557, "y": 167.286},
    {"x": 236.648, "y": 285.61},
    {"x": 90.929, "y": 326.412}
  ]
}

nested_checklist_annotation_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer",
      "classifications" : [
        {
          "name": "sub_checklist_question",
          "answer": {"name": "first_sub_checklist_answer"}
        }
      ]
  }]
}

text_annotation_ndjson = {
  "name": "free_text",
  "answer": "sample text",
}


##### 7.4. Create the label object

In [ ]:
# Create a Label object by identifying the applicable data row in Labelbox and providing a list of annotations
annotations = [
      radio_annotation_ndjson,
      checklist_annotation_ndjson,
      bbox_annotation_ndjson,
      bbox_with_radio_subclass_ndjson,
      polygon_annotation_ndjson,
      mask_annotation_ndjson,
      point_annotation_ndjson,
      polyline_annotation_ndjson,
      nested_radio_annotation_ndjson,
      nested_checklist_annotation_ndjson,
      text_annotation_ndjson
  ]
# gts = bbox_annotations
gts = (annotations)
ndjson_annotation = []
for data_row in dataset.export_data_rows():
  for annot in gts:
      ndjson_annotation.append({
        **annot,
        'uuid': str(uuid.uuid4()),
        'dataRow': {'id': data_row.uid}
      })



##### 7.5. Upload annotations to the project using Label Import

In [ ]:
upload_job_annotation = LabelImport.create_from_objects(
    client = client,
    project_id = project.uid,
    name="annotation_import_" + str(uuid.uuid4()),
    labels=ndjson_annotation)

upload_job_annotation.wait_until_done()
# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_annotation.errors)


##### 7.6 Send the annotations to the Model Run

In [ ]:
# get the labels id from the project
task = project.export_v2()
task.wait_till_done()
label_ids = [l["id"] for dr in task.result for l in dr["projects"][project.uid]["labels"]]
model_run.upsert_labels(label_ids)

## Optional deletions for cleanup


In [ ]:
# project.delete()
# dataset.delete()